In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()

In [ ]:
!pip install gensim

In [ ]:
from gensim.corpora import Dictionary
from gensim.utils import simple_tokenize

posts = [post for post in newsgroups['data']]
dict_voc = Dictionary([[word for word in simple_tokenize(post)] for post in posts]) 

In [ ]:
bow = [dict_voc.doc2bow(simple_tokenize(post)) for post in posts]

In [ ]:
from gensim.models.ldamulticore import LdaMulticore
lda = LdaMulticore(corpus=bow, id2word=dict_voc, chunksize=2000, iterations=400, num_topics=5, 
                   passes=20, eval_every=None, random_state=42)

In [ ]:
lda.print_topics()

In [ ]:
!pip install pyLDAvis==3.3.1

In [ ]:
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(lda, bow, dict_voc)
pyLDAvis.display(lda_display)

In [ ]:
import gensim

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.w2v || wget https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.w2v.gz && gunzip heise-articles-2020.w2v.gz")
    newsticker_w2v = 'heise-articles-2020.w2v'
else:
    newsticker_w2v = '../99_Common/heise-articles-2020.w2v'

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(newsticker_w2v, binary=False)

In [ ]:
w2v.most_similar("apple")

In [ ]:
w2v.most_similar(positive=["google", "ios"], negative=["apple"])